# Helper Methods

In [1]:
from sklearn import preprocessing
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64
import matplotlib.image as mpimg

train_directory = os.getcwd() + '/all/train'
test_directory = os.getcwd() + '/all/test' 


def exit():
    sys.exit()

# Gathering the image data

In [2]:
%%time


def get_dataset_df():  
    cameras = os.listdir(train_directory)
    
    train_images = []

    for camera in cameras:
        for fname in sorted(os.listdir(train_directory + '/' + camera)):
            train_images.append((camera, fname))

    train = pd.DataFrame(train_images, columns=['camera', 'fname'])
    all_classes = train['camera'].unique()
    
    test_images = []
    camera_name = 'Unknown'
    for image in os.listdir(test_directory):
        test_images.append((camera_name, image))

    test = pd.DataFrame(test_images, columns=['camera', 'fname'])
    test_images = os.listdir(test_directory) 
    return train, test

def _decode_image(filename):
    img = mpimg.imread(filename)
    return tf.cast(img, tf.float32)
    
#     image_string = tf.read_file(filename)
#     image_decoded = tf.image.decode_jpeg(image_string, channels=3)
#     image = tf.cast(image_decoded, tf.float32)
#     image_resized = tf.image.resize_images(image, [2000, 2000])

    
    return image

def load_image_data(df, offset=None, size=None):
    
    training_data = []

    
    list_of_images = ['{}/{}/{}'.format(train_directory, row['camera'], row['fname']) 
                      for index, row in df[offset:offset+size].iterrows()
                     ]
    
    init_op = tf.global_variables_initializer()
    
    target_height = 2432
    target_width = 4320
    

    with tf.Session() as sess:
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
   
        print "Processing %s images" % len(list_of_images)
        
        count, threshold = 0 , 10

        for i in list_of_images:
            
            
            try:
                original_image = _decode_image(i)

           
                image = tf.image.resize_image_with_crop_or_pad(
                    original_image,
                    target_height,
                    target_width
                )

                if count == threshold:
                    break            


                # Convert tensor to numpy array
                image_data = np.asarray(image.eval())
                
                training_data.append(image_data)
                
                count += 1

                
            except Exception as e:
                print str(e)
                print "Failed on %s" % count

 
        coord.request_stop()
        coord.join(threads)
        
    return training_data


train, test = get_dataset_df()


x_train_orig = load_image_data(train, offset=0, size=5)
y_train_orig = [row['camera'] for index, row in train[0:5].iterrows()]


x_test_orig = load_image_data(train, offset=11, size=5)
y_test_orig = [row['camera'] for index, row in train[6:12].iterrows()]

Processing 5 images
Processing 5 images
CPU times: user 20.2 s, sys: 10.9 s, total: 31.1 s
Wall time: 36.5 s


# The Model

In [2]:
import keras
import copy
import sys
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import matplotlib.pyplot as plt


batch_size = 5
num_classes = 1
epochs = 12

class_mapping = {}

def encode_classes():
    classes = os.listdir(train_directory)
    
    for index, row in enumerate(classes):
        class_mapping[row] = index
       
        
encode_classes()


x_train = np.array(copy.deepcopy(x_train_orig))
y_train = np.array(copy.deepcopy(y_train_orig))
x_test = np.array(copy.deepcopy(x_train_orig))
y_test = np.array(copy.deepcopy(y_train_orig))


print class_mapping
y_train = [class_mapping[x] for x in y_train]
y_test = [class_mapping[x] for x in y_test]


img_rows, img_cols = 2432, 4320
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 3)

    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print("Training samples: {}".format(x_train.shape[0]))
print("Test samples: {}".format(x_test.shape[0]))

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

NameError: global name 'os' is not defined

# Testing the Model by Fitting

In [ ]:
%%time 
import time

start_time = time.time()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=2,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 5 samples, validate on 5 samples
Epoch 1/1
